In [1]:
# import modules

import json
import pickle
import spacy
import string
import warnings
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
nlp = spacy.load("en_core_web_sm")
from keras.layers import LSTM,Embedding

Using TensorFlow backend.


In [2]:
#initializing lists and variables
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
type(words)

list

In [3]:
#preparing data
#converting user response to spacy token
def responsetotoken(user_response):
    return nlp(user_response)



#function to word tokenize and lower case
def lower_token(text):
  return [token.lower_ for token in text]

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # converting pattern to spacy token
        pattern=nlp(pattern)
        w=lower_token(pattern)
        # take each word and tokenize it

        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [4]:
#preprocessing 
## lemmatized lowercase words after removing punctuations with SPACY

def LemNormalize(text):
    token_text=responsetotoken(text)
    return [token.lower_ for token in token_text if not token.is_stop and not token.is_punct]

corpus=[]

for w in words:
    norm_word=(LemNormalize(w))
    corpus.extend(norm_word)


words = sorted(list(set(corpus)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

108 documents
20 classes [' Health and safety', 'assessment', 'business', 'cases', 'cases in canada', 'cerb', 'coronavirus', 'ei', 'goodbye', 'greeting', 'individual', 'noanswer', 'options', 'prevention', 'support', 'symptoms', 'thanks', 'travellers', 'update', 'video']
93 unique lemmatized words [' ', 'abcdef', 'affect', 'apply', 'area', 'assessment', 'awesome', 'benefits', 'benfits', 'busines', 'business', 'bye', 'canada', 'canadians', 'cases', 'cerb', 'chatting', 'confirmed', 'convid-19', 'coronavirus', 'count', 'country', 'covid-19', 'day', 'deaths', 'determinte', 'device', 'disease', 'drug', 'dumb', 'economic', 'ei', 'emergencey', 'employee', 'financial', 'good', 'goodbye', 'health', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'individual', 'individuals', 'insurance', 'later', 'live', 'lord', 'mad', 'measures', 'medical', 'monitoring', 'namastey', 'namstey', 'new', 'nice', 'notice', 'novel', 'offered', 'outbreak', 'pandameic', 'praise', 'prepared', 'prevent', 'prev

In [5]:
#saving pickle file


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [6]:
## lemmatized lowercase words after removing punctuations with SPACY
def LemNormalize_train(text):
    token_text=responsetotoken(text)
    return [token.lower_ for token in token_text if  not token.is_punct]


In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    #empty list for lemmatizing words in doc
    corpus=[]
    for w in pattern_words:
        norm_word=(LemNormalize_train(w))
        corpus.extend(norm_word)

    pattern_words = corpus
    print("pattern words:",pattern_words)
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    corpus=[]
    

pattern words: ['hi', 'there']
pattern words: ['how', 'are', 'you']
pattern words: ['is', 'anyone', 'there']
pattern words: ['hey']
pattern words: ['hola']
pattern words: ['hello']
pattern words: ['good', 'day']
pattern words: ['namstey']
pattern words: ['namastey']
pattern words: ['praise', 'the', 'lord']
pattern words: ['bye']
pattern words: ['see', 'you', 'later']
pattern words: ['goodbye']
pattern words: ['thanks', 'see', 'you', 'later']
pattern words: ['nice', 'chatting', 'to', 'you', 'bye']
pattern words: ['till', 'next', 'time']
pattern words: ['thanks']
pattern words: ['thank', 'you']
pattern words: ['that', "'s", 'helpful']
pattern words: ['awesome', 'thanks']
pattern words: ['thanks', 'for', 'helping', 'me']
pattern words: ['random']
pattern words: []
pattern words: ['anything']
pattern words: ['nothing']
pattern words: ['abcdef']
pattern words: [' ']
pattern words: ['are', 'you', 'dumb']
pattern words: ['mad']
pattern words: ['do', 'you', 'understand']
pattern words: ['are',

In [8]:
print(output_row)
print(pattern_words)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['travel', 'safety']


In [9]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
print(training)

Training data created
[[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
# Create model - 4 layers. First layer 128 neurons, second layer 64 neurons, third layer 32 nuerons  and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model_1.h5', hist)

print("model created")


Epoch 1/200
108/108 [==============================] - 1s 9ms/step - loss: 2.9979 - accuracy: 0.0556
Epoch 2/200
108/108 [==============================] - 0s 2ms/step - loss: 2.9429 - accuracy: 0.1204
Epoch 3/200
108/108 [==============================] - 0s 1ms/step - loss: 2.9086 - accuracy: 0.0833
Epoch 4/200
108/108 [==============================] - 0s 2ms/step - loss: 2.8844 - accuracy: 0.1111
Epoch 5/200
108/108 [==============================] - 0s 2ms/step - loss: 2.8480 - accuracy: 0.1019
Epoch 6/200
108/108 [==============================] - 0s 2ms/step - loss: 2.8264 - accuracy: 0.1481
Epoch 7/200
108/108 [==============================] - 0s 2ms/step - loss: 2.7624 - accuracy: 0.1111
Epoch 8/200
108/108 [==============================] - 0s 2ms/step - loss: 2.7483 - accuracy: 0.1389
Epoch 9/200
108/108 [==============================] - 0s 2ms/step - loss: 2.6140 - accuracy: 0.2037
Epoch 10/200
108/108 [==============================] - 0s 2ms/step - loss: 2.5345 - accur

108/108 [==============================] - 0s 2ms/step - loss: 0.3552 - accuracy: 0.8981
Epoch 80/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3743 - accuracy: 0.8889
Epoch 81/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3765 - accuracy: 0.8426
Epoch 82/200
108/108 [==============================] - 0s 2ms/step - loss: 0.5086 - accuracy: 0.8148
Epoch 83/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3506 - accuracy: 0.8796
Epoch 84/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3515 - accuracy: 0.8611
Epoch 85/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3624 - accuracy: 0.8519
Epoch 86/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3934 - accuracy: 0.8333
Epoch 87/200
108/108 [==============================] - 0s 2ms/step - loss: 0.4346 - accuracy: 0.8333
Epoch 88/200
108/108 [==============================] - 0s 2ms/step - loss: 0.4066 - accuracy: 

108/108 [==============================] - 0s 2ms/step - loss: 0.2586 - accuracy: 0.8981
Epoch 159/200
108/108 [==============================] - 0s 2ms/step - loss: 0.2935 - accuracy: 0.8519
Epoch 160/200
108/108 [==============================] - 0s 2ms/step - loss: 0.2483 - accuracy: 0.8889
Epoch 161/200
108/108 [==============================] - 0s 2ms/step - loss: 0.2875 - accuracy: 0.8519
Epoch 162/200
108/108 [==============================] - 0s 2ms/step - loss: 0.2953 - accuracy: 0.8704
Epoch 163/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3595 - accuracy: 0.8704
Epoch 164/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3726 - accuracy: 0.8519
Epoch 165/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3426 - accuracy: 0.8611
Epoch 166/200
108/108 [==============================] - 0s 2ms/step - loss: 0.4302 - accuracy: 0.8241
Epoch 167/200
108/108 [==============================] - 0s 2ms/step - loss: 0.3239 - a